### Parte 1: Descargas las paginas web de manera masiva

In [26]:
import requests
import os
import re

# Crear una carpeta de salida
os.makedirs("paginas_descargadas", exist_ok=True)

# Función para convertir URL en nombre de archivo válido
def url_a_nombre_archivo(url):
    nombre = re.sub(r'[^a-zA-Z0-9]', '_', url)
    return nombre[:100] + ".html"  # Limita longitud

with open("urls_recetas.txt", "r") as archivo:
    urls = [linea.strip() for linea in archivo if linea.strip()]

for url in urls:
    try:
        print(f"Descargando: {url}")
        #Agente para que la pagina no bloquee la solicitud
        headers = {
                "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
        }
        respuesta = requests.get(url, headers=headers, timeout=10)

        respuesta.raise_for_status()  # Lanza error si status ≠ 200

        nombre_archivo = url_a_nombre_archivo(url)
        ruta = os.path.join("paginas_descargadas", nombre_archivo)

        with open(ruta, "w", encoding="utf-8") as f:
            f.write(respuesta.text)
        print(f"✅ Guardado en: {ruta}")

    except Exception as e:
        print(f"❌ Error con {url}: {e}")


Descargando: https://www.allrecipes.com/recipe/20867/cheesy-zucchini-casserole-i/
✅ Guardado en: paginas_descargadas\https___www_allrecipes_com_recipe_20867_cheesy_zucchini_casserole_i_.html
Descargando: https://www.allrecipes.com/recipe/270940/air-fryer-eggplant-parmesan/
✅ Guardado en: paginas_descargadas\https___www_allrecipes_com_recipe_270940_air_fryer_eggplant_parmesan_.html
Descargando: https://www.allrecipes.com/recipe/237476/chef-johns-succotash/
✅ Guardado en: paginas_descargadas\https___www_allrecipes_com_recipe_237476_chef_johns_succotash_.html
Descargando: https://www.allrecipes.com/recipe/237805/baked-tomato-slices/
✅ Guardado en: paginas_descargadas\https___www_allrecipes_com_recipe_237805_baked_tomato_slices_.html
Descargando: https://www.allrecipes.com/recipe/106695/prosciutto-e-melone-italian-ham-and-melon/
✅ Guardado en: paginas_descargadas\https___www_allrecipes_com_recipe_106695_prosciutto_e_melone_italian_ham_and_melon_.html
Descargando: https://www.allrecipes.com


### Importar librerias

In [4]:
import os
from bs4 import BeautifulSoup
import pandas as pd

### Ciclo para descarrgar los elementos pargina por pagina y almacenarlos en un dataframe

In [5]:
# Ruta de la carpeta con las páginas descargadas
carpeta_paginas = "paginas_descargadas"

# Lista para almacenar los datos
datos = []

# Recorrer todos los archivos HTML en la carpeta
for archivo in os.listdir(carpeta_paginas):
    if archivo.endswith(".html"):
        ruta_archivo = os.path.join(carpeta_paginas, archivo)
        with open(ruta_archivo, "r", encoding="utf-8") as f:
            contenido = f.read()
            soup = BeautifulSoup(contenido, "html.parser")
            
            # Extraer el título de la página
            titulo = soup.title.string if soup.title else "Sin título"
            
            # Extraer el resumen
            resumen_tag = soup.find("p", class_="article-subheading text-utility-300")
            resumen = resumen_tag.text.strip() if resumen_tag else "Sin resumen"
            
            # Extraer la calificación
            rating_tag = soup.find("div", id="mm-recipes-review-bar__rating_1-0")
            rating = rating_tag.text.strip() if rating_tag else "Sin calificación"
            
            # Extraer el tiempo de cocción (Cook Time)
            cook_time_value = "Sin tiempo de cocción"
            items = soup.find_all("div", class_="mm-recipes-details__item")

            for item in items:
                label = item.find("div", class_="mm-recipes-details__label")
                if label and "Cook Time:" in label.text:
                    value = item.find("div", class_="mm-recipes-details__value")
                    if value:
                        cook_time_value = value.text.strip()
                    break

            # Extraer las porciones (Servings)
            servings_value = "Sin información de porciones"
            items = soup.find_all("div", class_="mm-recipes-details__item")

            for item in items:
                label = item.find("div", class_="mm-recipes-details__label")
                if label and "Servings:" in label.text:
                    value = item.find("div", class_="mm-recipes-details__value")
                    if value:
                        servings_value = value.text.strip()
                    break  # Se encontró, no hace falta seguir


            # Extraer lista de ingredientes
            ingredientes = []

            # Buscar la lista de ingredientes
            ingredientes_lista = soup.find("ul", class_="mm-recipes-structured-ingredients__list")

            if ingredientes_lista:
                items = ingredientes_lista.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
                for item in items:
                    cantidad = item.find("span", attrs={"data-ingredient-quantity": True})
                    unidad = item.find("span", attrs={"data-ingredient-unit": True})
                    nombre = item.find("span", attrs={"data-ingredient-name": True})
                    
                    ingrediente = {
                        "cantidad": cantidad.text.strip() if cantidad else "",
                        "unidad": unidad.text.strip() if unidad else "",
                        "nombre": nombre.text.strip() if nombre else ""
                    }
                    ingredientes.append(ingrediente)

            
            # Agregar los datos a la lista
            datos.append({
                "Título": titulo,
                "Resumen": resumen,
                "Calificación": rating,
                "Cook Time": cook_time_value,
                "Servings": servings_value,
                "Ingredientes": ingredientes
            })

# Crear un DataFrame con los datos
df = pd.DataFrame(datos)

In [6]:
df

,Título,Resumen,Calificación,Cook Time,Servings,Ingredientes
0,Best Smoothies for Pre & Post Workouts,Sin resumen,Sin calificación,Sin tiempo de cocción,Sin información de porciones,[]
1,How to Make Banana Bread,Banana bread is one of the most popular recipe...,Sin calificación,Sin tiempo de cocción,Sin información de porciones,[]
2,How to Make High-Protein Smoothies to Fuel You...,Sin resumen,Sin calificación,Sin tiempo de cocción,Sin información de porciones,[]
3,How to Make A Healthy Smoothie,Whether you're on the hunt for a healthy recip...,Sin calificación,Sin tiempo de cocción,Sin información de porciones,[]
4,Best Banana Breads,Sin resumen,Sin calificación,Sin tiempo de cocción,Sin información de porciones,[]
...,...,...,...,...,...,...
108,White Cheese Chicken Lasagna Recipe,Chicken lasagna with spinach and a creamy whit...,4.4,50 mins,12,"[{'cantidad': '9', 'unidad': '', 'nombre': 'la..."
109,Broccoli Chicken Divan Recipe,Chicken Divan with broccoli is a quick and eas...,4.4,20 mins,6,"[{'cantidad': '1', 'unidad': 'pound', 'nombre'..."
110,Broccoli Chicken Casserole Recipe,"A deliciously creamy chicken, broccoli, and st...",4.2,40 mins,4,"[{'cantidad': '4', 'unidad': '(5 ounce)', 'nom..."
111,Baked Teriyaki Chicken Recipe,This teriyaki chicken recipe uses a homemade t...,4.7,55 mins,6,"[{'cantidad': '½', 'unidad': 'cup', 'nombre': ..."


### Aplicar un Rag Conversacional

In [31]:
from dotenv import load_dotenv
import os

In [34]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
import pandas as pd

# Cargar el DataFrame (ya generado en tu código)
df = pd.DataFrame(datos)  

# Convertir el DataFrame a texto para indexarlo
def dataframe_to_text(df):
    textos = []
    for _, row in df.iterrows():
        texto = f"Título: {row['Título']}\nResumen: {row['Resumen']}\nCalificación: {row['Calificación']}\n"
        texto += f"Cook Time: {row['Cook Time']}\nServings: {row['Servings']}\nIngredientes: {row['Ingredientes']}\n"
        textos.append(texto)
    return textos

textos = dataframe_to_text(df)


# Asegúrate de no estar usando GOOGLE_API_KEY aquí
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = FAISS.from_texts(textos, embeddings)

In [36]:
# Crear el sistema RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=OPENAI_API_KEY),
    retriever=vectorstore.as_retriever()
)

In [ ]:
# Hacer preguntas al sistema
pregunta = "¿Que calificacion tiene la receda Baked Teriyaki Chicken Recipe?"
respuesta = qa_chain.run(pregunta)
print(respuesta)

  4.7
